In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn
import random


In [50]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [51]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [52]:
filename = 'Dataset/Sensor_Discrimination_normalized.csv'
df=pd.read_csv(filename)
# shuffle the DataFrame rows
df = df.sample(frac = 1) #frac = fraction
dataset=df.iloc[:,:-1]
label=df.iloc[:,-1]
print(dataset)
print(label)

            F2        F3        F1       F11        F4        F6       F10  \
193   0.409412  0.515310  0.266601  0.168599  0.426576  0.029192  0.177861   
1509  0.288556  0.835296  0.722233  0.409043  0.914186  0.492303  0.950301   
1127  0.261604  0.964670  0.921645  0.399113  0.987028  0.580264  0.941453   
2191  0.188257  0.904441  0.707738  0.220390  0.746217  0.323529  0.292357   
405   0.676791  0.480485  0.258570  0.184574  0.402129  0.011407  0.186822   
...        ...       ...       ...       ...       ...       ...       ...   
3     0.476578  0.529778  0.314398  0.188298  0.471312  0.099313  0.215361   
910   0.895187  0.870962  0.857395  0.253014  0.963745  0.561297  0.763931   
1207  0.957219  0.926649  0.823702  0.204078  0.864627  0.399670  0.415098   
300   0.792513  0.469886  0.264643  0.173582  0.356893  0.030539  0.181777   
267   0.124299  0.925639  0.912243  0.936170  0.970231  0.747938  0.939006   

            F9        F5      F7     F12       F8  
193   0.033

In [53]:
pcaTrain = PCA(n_components=3).fit(dataset)# --train-- pca with the "dataset" Dataframe
print(pcaTrain)
compressed_dataset_X= pcaTrain.transform(dataset) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X))
print(len(compressed_dataset_X[0]))
print(compressed_dataset_X)

PCA(n_components=3)
<class 'numpy.ndarray'>
3
[[ 1.84953354 -0.87269856  0.23445507]
 [ 1.46823148  0.3490067  -0.13801831]
 [-3.18630691  1.48962021  0.6166575 ]
 ...
 [-2.93581681 -0.16348927  0.25466583]
 [-3.68048696 -1.26097346  0.35123575]
 [-3.09720773  0.85575057 -0.23379201]]


In [54]:
dataset_arr=dataset.to_numpy()


In [55]:
# from NecessaryModules.splitData import split_data
X_train, X_test, y_train, y_test = train_test_split(dataset_arr, label, test_size=0.3, random_state=42)
print(len(X_train))

1548


In [56]:
compressed_dataset_X_train= pcaTrain.transform(X_train) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train))
print(len(compressed_dataset_X_train))
print(compressed_dataset_X_train)

compressed_dataset_X_test= pcaTrain.transform(X_test) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test))
print(len(compressed_dataset_X_test))
print(compressed_dataset_X_test)

<class 'numpy.ndarray'>
1548
[[-2.9013479   0.32935409  0.07955765]
 [ 1.56152952 -0.30192325 -0.31674257]
 [ 1.18785917 -0.77812526  0.1356577 ]
 ...
 [-3.17855277  1.08685607  0.20928067]
 [-3.93844722 -0.92774663  0.00628211]
 [ 1.22680988 -0.61820096 -0.17317756]]
<class 'numpy.ndarray'>
664
[[ 1.57117946 -0.50825339  0.07986403]
 [ 0.83528177 -0.92018484  0.29158838]
 [ 1.75580005 -0.78940411  0.39461333]
 ...
 [-3.33792142  1.68908457  0.24686796]
 [ 1.38546371 -0.44951489  0.01597519]
 [-3.87877118 -0.98217673  0.10734265]]


# tune XG-Boost

In [57]:
# train the model with random parameter

# Preparing the regressor and fitting data with some parameters
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 1, 
                          learning_rate = 0.3,
                          max_depth = 15, 
                          alpha = 10, 
                          n_estimators = 100, 
                          verbose=1, 
                          min_child_weight = 1, 
                          colsample_bylevel = 1, 
                          reg_alpha = 2) 

xg_reg.fit(compressed_dataset_X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=1, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.3, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [58]:
# Finding mean absolute error
preds = xg_reg.predict(compressed_dataset_X_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.42018072289156627

In [59]:
# Defining the best parameter with optuna
from sklearn.model_selection import KFold
K = 5 # 5 fold cross validation
kf = KFold(n_splits = K, random_state = 42, shuffle = True)


train_tunned = np.array(compressed_dataset_X)
target_train_tunned = np.array(label)

# test = np.array(X)
# target_train = df_train['congestion'].values

# test_preds = np.zeros((len(test)))
oof_preds = np.zeros((len(train_tunned)))
abs_error_arr=[]
def objective(trial):

 for train_index, val_index in (kf.split(compressed_dataset_X)):
    train_X, valid_X = train_tunned[train_index], train_tunned[val_index]
    train_Y, valid_Y = target_train_tunned[train_index], target_train_tunned[val_index]
    
        
        # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=0)
    param = {
            # 'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process 
            'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [1e-5,1e-4,1e-3,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.04,0.08,0.12,0.20, 0.24, 0.30, 0.38, 0.42, 0.48, 0.50, 0.60, 0.70, 0.80, 0.90, 1.0]),
            'n_estimators': trial.suggest_int('n_estimators', 1, 300),
            'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20,25,30,35,40]),
            'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        }
    model = xgb.XGBRegressor(**param)  
        
    model.fit(train_X,train_Y,eval_set=[(valid_X,valid_Y)],early_stopping_rounds=100,verbose=False)
        
    preds = model.predict(valid_X)
    oof_preds[val_index] = preds # preds gives total 3846 predicted value for valid_X


 
#  print(len(oof_preds))
      
 mae = mean_absolute_error(target_train_tunned, oof_preds)
        
 return mae

In [60]:
# this is the main optuna for optimizing a object-->"objective"
study = optuna.create_study(direction='minimize',study_name="XG_tune")

[I 2024-02-12 11:12:53,176] A new study created in memory with name: XG_tune


In [61]:

study.optimize(objective, n_trials=100) # 150 bar "objective" object ke trial kore kore dekhbe, here total 5*100=750
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-02-12 11:12:54,628] Trial 0 finished with value: 0.2734728475441091 and parameters: {'lambda': 0.022322473550623327, 'alpha': 1.0201035196451722, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.4, 'subsample': 0.8, 'learning_rate': 0.01, 'n_estimators': 256, 'max_depth': 15, 'random_state': 48, 'min_child_weight': 130}. Best is trial 0 with value: 0.2734728475441091.
[I 2024-02-12 11:12:55,273] Trial 1 finished with value: 0.5158673265096723 and parameters: {'lambda': 0.03818769212443581, 'alpha': 1.018222421872955, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.4, 'learning_rate': 0.01, 'n_estimators': 180, 'max_depth': 35, 'random_state': 24, 'min_child_weight': 149}. Best is trial 0 with value: 0.2734728475441091.
[I 2024-02-12 11:12:56,178] Trial 2 finished with value: 0.271803153278584 and parameters: {'lambda': 0.02508730969042275, 'alpha': 4.958425745386568, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.7, 'subsample': 0.8, 'learning_rate': 0.48, 'n_

Number of finished trials: 100
Best trial: {'lambda': 0.8853639317687356, 'alpha': 0.7469283203638263, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.5, 'subsample': 0.8, 'learning_rate': 0.08, 'n_estimators': 129, 'max_depth': 15, 'random_state': 48, 'min_child_weight': 1}


In [62]:
from matplotlib import pyplot as plt
fig_study = optuna.visualization.plot_parallel_coordinate(study, params=["lambda", "alpha","colsample_bytree","colsample_bylevel","subsample","learning_rate","n_estimators","max_depth","random_state","min_child_weight"])
fig_study.show()

In [63]:
#evaluate the trial
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  0.10028135822133977
  Params: 
    lambda: 0.8853639317687356
    alpha: 0.7469283203638263
    colsample_bytree: 1.0
    colsample_bylevel: 0.5
    subsample: 0.8
    learning_rate: 0.08
    n_estimators: 129
    max_depth: 15
    random_state: 48
    min_child_weight: 1


# fit the XG-Boost

In [64]:
from sklearn.model_selection import KFold
K = 5 # 5 fold cross validation
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [65]:
# Let us make predictions for each of the 5 models and find mean 
# of those predictions

from tqdm import tqdm
train = np.array(compressed_dataset_X_train)
target_train = np.array(y_train)

test = np.array(compressed_dataset_X_test)
# target_train = df_train['congestion'].values

test_preds = np.zeros((len(test)))
oof_preds = np.zeros((len(train)))
for train_index, val_index in tqdm(kf.split(compressed_dataset_X_train), total=5):
    train_X, valid_X = train[train_index], train[val_index]
    train_y, valid_y = target_train[train_index], target_train[val_index]
    xgb_params = study.best_trial.params
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(train_X, train_y,eval_set=[(valid_X,valid_y)],early_stopping_rounds=100,verbose=False)
    # for train accuracy
    preds = model.predict(valid_X)
    oof_preds[val_index] = preds
    # for test accuracy
    preds = model.predict(test)
    test_preds += (preds)/5
print(np.abs(oof_preds-target_train).mean())

100%|██████████| 5/5 [00:01<00:00,  2.95it/s]

0.09971808568987056


In [66]:
target_test = np.array(y_test)

In [67]:
from sklearn.metrics import accuracy_score
# valid_Acc_SVC=accuracy_score(target_train,oof_preds)
# print(valid_Acc_SVC)

# another technique for calculate test accuracy
test_Acc_SVC=accuracy_score(target_test,test_preds.round())
print(test_Acc_SVC)

0.9307228915662651


In [68]:
valid_Acc=accuracy_score(target_train,oof_preds.round())
print(valid_Acc)

0.9483204134366925


In [69]:
train_Acc=model.score(compressed_dataset_X_train,y_train)
print(train_Acc)

0.9690215447838055


In [70]:
test_Acc=model.score(compressed_dataset_X_test,y_test)
print(test_Acc)

0.8973250364738544
